In [1]:
print('qwen2-inference-demo')

qwen2-inference-demo


In [2]:
!pip install sagemaker==2.224.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.6 MB/s eta 0:00:0000:01
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.219.0
    Uninstalling sagemaker-2.219.0:
      Successfully uninstalled sagemaker-2.219.0


In [3]:
import sagemaker
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

print(region)
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
ap-southeast-1
arn:aws:iam::826531593231:role/service-role/AmazonSageMaker-ExecutionRole-20240630T232601


In [4]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [5]:
from string import ascii_uppercase
from random import choice

def generate_random_label():
    letters = ascii_uppercase
   
    return ''.join(choice(letters) for i in range(10))

In [6]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface",
  region=region,
  version="2.0.2"
)

model_name = "Qwen2-" + generate_random_label()

In [7]:
image_uri

'763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.0.2-gpu-py310-cu121-ubuntu22.04'

In [8]:
model_name

'Qwen2-HIHBPNYSFX'

In [9]:
hub_env = {
  'HF_MODEL_ID': 'Qwen/Qwen2-0.5B-Instruct',
  'HF_TASK': 'text-generation',
  'SM_NUM_GPUS': '1'
}

In [10]:
model = HuggingFaceModel(
    name=model_name,
    env=hub_env,
    role=role,
    image_uri=image_uri
)

In [11]:
predictor = model.deploy(
  initial_instance_count=1,
  instance_type="ml.g4dn.xlarge",
  endpoint_name=model_name,
)

--------!

In [12]:
result = predictor.predict({
    "inputs": "My name is Clara and I am",
})
print(result)

[{'generated_text': 'My name is Clara and I am a part of the team at Alla Chichi Premium Tomato Bombs. Here at Ellie Mae we are providing clothing and accessories for outdoor enthusiasts of all ages through the season. We rely on our excellent knowledge to take you there to the top of the mountain to make a fantastic experience. Let’s start!\nWe are passionate about our work and creating amazing experiences for everyone. Whether you are the club president, girlfriend, aunt, or someone just visiting, we enjoy the excitement of their first trip! If you'}]


In [13]:
predictor.delete_model()
predictor.delete_endpoint()